In [104]:
import numpy as np
import random
import math
%load_ext watermark

random.seed(1234)

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [105]:
%watermark

Last updated: 2024-03-18T10:01:43.144597-04:00

Python implementation: CPython
Python version       : 3.12.2
IPython version      : 8.20.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 11
Machine     : AMD64
Processor   : Intel64 Family 6 Model 141 Stepping 1, GenuineIntel
CPU cores   : 12
Architecture: 64bit



In [106]:
%watermark --iversions

numpy: 1.26.4



# This notebook contains python implementatations of the code in The Scientist and Engineer's Guide to Digital Signal Processing" by By Steven W. Smith, Ph.D.  I've tried to have two versions of the code.  The first version is a almost exact translation of the BASIC code into Python, which will result in bad/low-performance/ugly python code (e.g. looping everything).  I've also tried to reimplement the code by being more pythonic and also using the libraries that would normally be used when implementing these types of functions.   

# I've tried to avoid using numpy or scipy because the goal of the example code is to show and explain how some of these DSP functions are actually implemented.  In any non-academic setting it would be more pragmatic to use a numpy array and not a python list, but I think it's easier to stick to the spirit of the original BASIC code using standard python data structures and functions.

# Chapter 2

## 2-1 Calculate Mean and Standard Deviation

In [107]:
#2-1 BASIC to Python
n = 512
#Generate random integers
x = [random.randint(0,3) for i in range(n)]

mean=0
for i in range(512):
    mean = mean + x[i]
mean = mean/n

variance = 0
for i in range(512):
    variance = variance + (x[i] - mean)**2
sd = math.sqrt(variance/(n-1))
print(f'Mean:{round(mean,2)},Standard Deviation:{round(sd,2)}')

Mean:1.54,Standard Deviation:1.11


In [108]:
#2-1 Pythonic
#n = 512
#x = [random.randint(0,3) for i in range(n)]

mean = sum(x)/n
variance = sum([(i - mean)**2 for i in x])
sd = math.sqrt(variance/(n-1))
print(f'Mean:{round(mean,2)},Standard Deviation:{round(sd,2)}')

Mean:1.54,Standard Deviation:1.11


## 2-2 Calculate Running Mean and Standard Deviation

In [109]:
#2-2 BASIC To Python
#Generate random integers
x = [random.randint(0,3) for i in range(10)]
n= 0               
total = 0 #Sum is a python function, so we cannot use it as a variable name
sum_squares = 0

#Since this is a "running" or streaming calcualation, the loop simulates values arriving one after another, and then the statistics are calculated for every new value
for i in range(len(x)):
    n = n+1         
    total = total + x[i]
    sum_squares = sum_squares + x[i]**2
    mean = total/n
    #I needed to add this if statement because when n is 1, the denominator for the variance calcualation is zero. This is listed as errata on the book website. 
    if n > 1:
        variance = (sum_squares - total**2/n) / (n-1)
    else:
        variance = (sum_squares - total**2/n) / (n)
    sd = math.sqrt(variance)
    print(f'Mean:{round(mean,2)},Standard Deviation:{round(sd,2)}')

Mean:3.0,Standard Deviation:0.0
Mean:1.5,Standard Deviation:2.12
Mean:1.67,Standard Deviation:1.53
Mean:1.5,Standard Deviation:1.29
Mean:1.8,Standard Deviation:1.3
Mean:2.0,Standard Deviation:1.26
Mean:1.86,Standard Deviation:1.21
Mean:1.88,Standard Deviation:1.13
Mean:1.78,Standard Deviation:1.09
Mean:1.9,Standard Deviation:1.1


In [110]:
#2-2 Pythonic.              
total = 0
sum_squares = 0

for i,value in enumerate(x):        
    n = i + 1
    total = total + value
    sum_squares = sum_squares + value**2
    mean = total/(n)
    #I needed to add this if statement because when n is 1, the denominator for the variance calcualation is zero.  This is listed as errata on the book website. 
    if n > 1:
        variance = (sum_squares - total**2/(n)) / (n-1)
    else:
        variance = (sum_squares - total**2/(n)) / (n)
    sd = math.sqrt(variance)
    print(f'Mean:{round(mean,2)},Standard Deviation:{round(sd,2)}')

Mean:3.0,Standard Deviation:0.0
Mean:1.5,Standard Deviation:2.12
Mean:1.67,Standard Deviation:1.53
Mean:1.5,Standard Deviation:1.29
Mean:1.8,Standard Deviation:1.3
Mean:2.0,Standard Deviation:1.26
Mean:1.86,Standard Deviation:1.21
Mean:1.88,Standard Deviation:1.13
Mean:1.78,Standard Deviation:1.09
Mean:1.9,Standard Deviation:1.1


## 2-3 Calculate the Histogram, mean, and standard deviations

In [111]:
#2-3 BASIC To Python
n = 25001
#Generate random integers
x = [random.randint(0,254) for i in range(n)]

#Initialise list of histogram values
h = []
for i in range(255):
    h.append(0)

#Calculate Histogram
for i in range(len(x)):
    h[x[i]] = h[x[i]] + 1

mean = 0
for i in range(len(h)):
    mean = mean + i * h[i]
mean = mean / n

variance = 0
for i in range(len(h)):
    variance = variance + h[i] * (i - mean)**2
variance = variance / (n-1)
sd = math.sqrt(variance)
print(f'Mean:{round(mean,2)},Standard Deviation:{round(sd,2)}')

Mean:127.69,Standard Deviation:73.95


In [112]:
#2-3 Pythonic
n = 25001

#Initialise list of histogram values
h = [0] * 255

#Calculate Histogram
for i in range(len(x)):
    h[x[i]] = h[x[i]] + 1

#This is probably a great example of where numpy multiplication makes more sense, but we will avoid it and use a list comprehension
mean = sum([i * value for i,value in enumerate(h)])
mean = mean / n

variance = sum([value * (i - mean)**2 for i,value in enumerate(h)])
variance = variance / (n - 1)
sd = math.sqrt(variance)

print(f'Mean:{round(mean,2)},Standard Deviation:{round(sd,2)}')

Mean:127.69,Standard Deviation:73.95


## 2-4 Calculating the Binned Histogram

In [113]:
#2-4 BASIC To Python
n = 25000
#Generate random integers
x = [random.random()*10 for i in range(n)]

#Initialise list of histogram values.  Note that the length of the array is 1000 since so that position 999 exists.
h = []
for i in range(1000):
    h.append(0)

for i in range(n):
    bin_num = int(x[i] * 100) #Multipled by 100 instead of the .01 in the book.  This is listed as errata on the book website. 
    h[bin_num] = h[bin_num]+1

In [114]:
#2-4 Pythonic
#Initialise list of histogram values.  Note that the length of the array is 1000 since so that position 999 exists.
h = [0] * 1000

bin_num = [int((i) * 100) for i in x]
#h = [for i in bin_num]

#This avoids the loop, but I'm not confident that this is better than the loop.
from collections import Counter
bin_num_unsort = Counter(bin_num)
bin_num_sort = sorted(bin_num_unsort.items())
bin_num = [y for x,y in bin_num_sort]

# Chapter 6

# 6-1 Convolution using the input side algorithm

In [115]:
#6-1 BASIC To Python
x = [random.randint(0,3) for i in range(80)]
h = [random.randint(0,3) for i in range(30)]

y = []
for i in range(110):
    y.append(0)

for i in range(len(x)):
    for j in range(len(h)):
        y[i+j] = y[i+j] + x[i]*h[j]
print(y)

[1, 3, 4, 3, 2, 5, 8, 11, 15, 19, 18, 17, 24, 28, 29, 32, 37, 35, 44, 39, 42, 44, 37, 40, 41, 43, 50, 34, 55, 41, 41, 45, 35, 34, 44, 55, 52, 53, 46, 37, 41, 37, 38, 34, 47, 40, 37, 58, 45, 30, 43, 34, 39, 36, 44, 49, 55, 39, 39, 36, 53, 37, 37, 46, 57, 42, 47, 47, 37, 44, 57, 51, 51, 52, 51, 45, 53, 42, 39, 54, 53, 37, 56, 50, 51, 44, 42, 30, 46, 38, 24, 24, 41, 34, 31, 20, 22, 22, 20, 4, 6, 14, 14, 6, 3, 9, 15, 12, 6, 0]


In [116]:
#6-1 Pythonic
y = [0] * 110

for i, x_val in enumerate(x):
    for j, h_val in enumerate(h):
        y[i+j] = y[i+j] + x_val*h_val
print(y)

[1, 3, 4, 3, 2, 5, 8, 11, 15, 19, 18, 17, 24, 28, 29, 32, 37, 35, 44, 39, 42, 44, 37, 40, 41, 43, 50, 34, 55, 41, 41, 45, 35, 34, 44, 55, 52, 53, 46, 37, 41, 37, 38, 34, 47, 40, 37, 58, 45, 30, 43, 34, 39, 36, 44, 49, 55, 39, 39, 36, 53, 37, 37, 46, 57, 42, 47, 47, 37, 44, 57, 51, 51, 52, 51, 45, 53, 42, 39, 54, 53, 37, 56, 50, 51, 44, 42, 30, 46, 38, 24, 24, 41, 34, 31, 20, 22, 22, 20, 4, 6, 14, 14, 6, 3, 9, 15, 12, 6, 0]


# 6-1 Convolution using the output side algorithm

In [117]:
#6-2 BASIC To Python
x = [random.randint(0,3) for i in range(80)]
h = [random.randint(0,3) for i in range(30)]

y = []
for i in range(110):
    y.append(0)
    for j in range(len(h)):
        #Python doesn't have a goto statement (for good reason!), so it doesn't make sense to try to recreate the original logic using if and go statements. 
        if ((i - j) >= 0) & ((i - j) < len(x)):
            y[i] = y[i] + h[j] * x[i-j]
print(y)

[0, 0, 3, 2, 8, 14, 14, 18, 11, 21, 26, 20, 24, 26, 32, 31, 32, 34, 39, 46, 48, 57, 58, 53, 69, 64, 58, 77, 70, 64, 71, 68, 64, 73, 69, 67, 74, 71, 53, 58, 74, 58, 60, 70, 58, 51, 56, 53, 53, 56, 55, 61, 45, 43, 40, 51, 47, 53, 53, 57, 46, 58, 50, 67, 67, 69, 72, 76, 58, 67, 66, 78, 71, 79, 82, 74, 68, 86, 75, 88, 87, 80, 70, 74, 59, 67, 57, 67, 54, 65, 57, 53, 42, 50, 39, 41, 35, 40, 30, 27, 28, 22, 19, 23, 20, 13, 13, 5, 3, 0]


In [118]:
#6-2 Pythonic

y = [0] * 110
for i in range(len(y)):
    for j in range(len(h)):
        #Python doesn't have a goto statement (for good reason!), so it doesn't make sense to try to recreate the original if and go statements. 
        if ((i - j) >= 0) & ((i - j) < len(x)):
            y[i] = y[i] + h[j] * x[i-j]
print(y)

[0, 0, 3, 2, 8, 14, 14, 18, 11, 21, 26, 20, 24, 26, 32, 31, 32, 34, 39, 46, 48, 57, 58, 53, 69, 64, 58, 77, 70, 64, 71, 68, 64, 73, 69, 67, 74, 71, 53, 58, 74, 58, 60, 70, 58, 51, 56, 53, 53, 56, 55, 61, 45, 43, 40, 51, 47, 53, 53, 57, 46, 58, 50, 67, 67, 69, 72, 76, 58, 67, 66, 78, 71, 79, 82, 74, 68, 86, 75, 88, 87, 80, 70, 74, 59, 67, 57, 67, 54, 65, 57, 53, 42, 50, 39, 41, 35, 40, 30, 27, 28, 22, 19, 23, 20, 13, 13, 5, 3, 0]
